In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
DirPpath = Path(os.path.abspath('')).parent # Fetching the current directory path - Specific for ipynb file - For .py: Path(os.path.dirname(os.path.realpath(__file__)).replace("\\", "/"))

PledgesCsvPath = str(DirPpath.absolute()) + "/CleanedData.csv"  

In [3]:
PledgesDf = pd.read_csv(PledgesCsvPath, index_col=0)

In [4]:
import string
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [5]:
def first_clean(text):
    return " ".join(text.split()).replace("_x000D_","")

first_clean(PledgesDf.iloc[3,1]) # Delete the returns from excel

'Achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity, respecting the socio-cultural authenticity of host communities.  CNA Turismo e Commercio, over the three-year period (between autumn 2022 and throughout 2023), will organize training seminars for businesses - with the involvement of public and private stakeholders - aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity.'

In [6]:
def preprocess(text):
    text = text.lower() # Lowercase all the characters from the string
    text = text.strip() # Remove the leading and trailing whitespaces
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) # Removing Punctuation
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text)) # Remove non alphanumeric characters
    text = re.sub(r'\d', '', text) # Removing digits
    text = re.sub(r'\s+', ' ', str(text).strip()) # Replacing "double, triple, etc" whitespaces by one
    return text

preprocess(first_clean(PledgesDf.iloc[3,1]))

'achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity respecting the socio cultural authenticity of host communities cna turismo e commercio over the three year period between autumn and throughout will organize training seminars for businesses with the involvement of public and private stakeholders aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity'

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [7]:
def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

stopword(preprocess(first_clean(PledgesDf.iloc[3,1])))

'achieve sustainable flexible solutions multimodal transport develop policies protect natural heritage biodiversity respecting socio cultural authenticity host communities cna turismo e commercio three year period autumn throughout organize training seminars businesses involvement public private stakeholders aimed implementation concrete solutions development good practices supply multimodal transport protocols respect protection natural heritage biodiversity'

In [8]:
# LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()


# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(nltk.word_tokenize(string))  # Get position tags
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in
         enumerate(word_pos_tags)]  # Map the position tag and lemmatize the word/token
    return " ".join(a)

lemmatizer(stopword(preprocess(first_clean(PledgesDf.iloc[3,1]))))

'achieve sustainable flexible solution multimodal transport develop policy protect natural heritage biodiversity respect socio cultural authenticity host community cna turismo e commercio three year period autumn throughout organize training seminar business involvement public private stakeholder aim implementation concrete solution development good practice supply multimodal transport protocol respect protection natural heritage biodiversity'

In [9]:
def preprocessing(text):

    global n
    n = n +1
    
    print("**************")
    print("n is : ")
    print(n)
    print("length of the text is : ")
    print(len(first_clean(text)))
    return lemmatizer(stopword(preprocess(first_clean(text))))

In [10]:
n=0
print("begin pre-processing")
PledgesDf['clean_text'] = PledgesDf['Pledge'].apply(lambda x: preprocessing(x))
print("end pre-processing")

begin pre-processing
**************
n is : 
1
length of the text is : 
1461
**************
n is : 
2
length of the text is : 
953
**************
n is : 
3
length of the text is : 
273
**************
n is : 
4
length of the text is : 
613
**************
n is : 
5
length of the text is : 
249
**************
n is : 
6
length of the text is : 
304
**************
n is : 
7
length of the text is : 
1125
**************
n is : 
8
length of the text is : 
640
**************
n is : 
9
length of the text is : 
352
**************
n is : 
10
length of the text is : 
130
**************
n is : 
11
length of the text is : 
2676
**************
n is : 
12
length of the text is : 
266
**************
n is : 
13
length of the text is : 
2442
**************
n is : 
14
length of the text is : 
361
**************
n is : 
15
length of the text is : 
2586
**************
n is : 
16
length of the text is : 
1031
**************
n is : 
17
length of the text is : 
1046
**************
n is : 
18
length of the text i

In [11]:
PledgesDf.head()

,Topic,Pledge,clean_text
0,1,HOTREC calls for a level playing field and fai...,hotrec call level play field fair competition ...
1,1,Actually we as an association are still pretty...,actually association still pretty much begin d...
2,1,Estonia sees the need to synchronize and harmo...,estonia see need synchronize harmonize rule sh...
3,2,Achieve sustainable and flexible solutions for...,achieve sustainable flexible solution multimod...
5,3,The Austrian Hoteliers Association (ÖHV) publi...,austrian hotelier association öhv publishes st...


In [12]:
pd.Series(" ".join(PledgesDf['clean_text']).split()).value_counts()[:30].index.tolist()

['tourism',
 'sustainable',
 'destination',
 'development',
 'data',
 'support',
 'project',
 'sustainability',
 'sector',
 'tourist',
 'develop',
 'digital',
 'use',
 'eu',
 'work',
 'action',
 'local',
 'service',
 'green',
 'plan',
 'city',
 'target',
 'year',
 'european',
 'strategy',
 'travel',
 'also',
 'regional',
 'share',
 'least']

In [12]:
tokens = [nltk.word_tokenize(i) for i in PledgesDf["clean_text"]]

In [13]:
word_freq = {}
for sent in tokens:
    for i in sent:

        if i not in word_freq.keys():
            word_freq[i] = 1
        else:
            word_freq[i] += 1
len(word_freq)

3267

In [14]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['tourism',
 'sustainable',
 'destination',
 'development',
 'data',
 'support',
 'project',
 'sustainability',
 'sector',
 'tourist']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

tfidf_wm = tfidfvectorizer.fit_transform(PledgesDf["clean_text"])

tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

### Visualiser les mots avec PCA ou t-sn

In [30]:
import sklearn
from sklearn import decomposition

In [31]:
X = model.wv.vectors

In [45]:
import matplotlib.pyplot as plt

In [54]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns

In [60]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 100), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=18).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    

In [19]:
import numpy as np

In [20]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [24]:
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)

# converting text to numerical data using Word2Vec
vectors_w2v = modelw.transform(tokens)

print(vectors_w2v)

[[-0.12560223  0.23184161  0.15681297 ... -0.2639868   0.09877244
  -0.09417447]
 [-0.14467603  0.2658919   0.18210404 ... -0.3050768   0.11311968
  -0.10659759]
 [-0.133623    0.25393614  0.17204833 ... -0.29037616  0.10742884
  -0.10489351]
 ...
 [-0.17307088  0.32190362  0.21772222 ... -0.36497992  0.13521887
  -0.12980379]
 [-0.15961476  0.29458204  0.20010252 ... -0.3362451   0.12484045
  -0.11860909]
 [-0.14230888  0.2624757   0.18064186 ... -0.30391672  0.11278331
  -0.10682324]]


### Visualiser les documents avec PCA

In [43]:
import gensim.downloader as api

In [45]:
wv2 = api.load('word2vec-google-news-300')

In [ ]:
wv2.most_similar(positive=["hotrec"])

In [65]:
w2v2 = dict(zip(wv2.index_to_key, wv2.vectors))
modelw2 = MeanEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens)

print(vectors_w2v2)

[[-0.06275298  0.01186308 -0.01152449 ... -0.01491957 -0.0153909
  -0.01533057]
 [-0.01571469 -0.00829576 -0.00121289 ... -0.00275593  0.01694754
  -0.01605253]
 [-0.02050312  0.01648125  0.00840877 ... -0.03087616  0.02504554
   0.02424622]
 ...
 [-0.05052965  0.01950734 -0.00666775 ... -0.02613797  0.05620015
   0.00589074]
 [-0.01669904  0.01115993 -0.00876993 ... -0.03272089  0.10013308
  -0.01204444]
 [-0.04008275  0.00388049 -0.01519181 ... -0.0157918   0.02797811
   0.02045738]]


In [20]:
#building Word2Vec model
class TfIdfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self

    def transform(self, X, tfidf):

        DocList = []
        i = 0
        
        for words in X:

            WordList = []

            for w in words:
                 
                try:
                    if w in self.word2vec:
                        weight = tfidf[w].iloc[i]
                        WordList.append(self.word2vec[w] * weight)
                    else:
                        WordList.append(np.zeros(self.dim))
                except:
                    WordList.append(np.zeros(self.dim))

            i+=1
            DocList.append(np.sum(np.array(WordList), axis = 0))
                         
        
        
        
        
        return np.array(DocList)

In [22]:
import numpy as np

In [23]:
w2v2 = dict(zip(wv2.index_to_key, wv2.vectors))
modelw2 = TfIdfEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens, df_tfidfvect)

print(vectors_w2v2)

[[-0.27269533  0.95840782 -1.35918353 ... -0.24890735 -1.48564479
   1.47997892]
 [-0.22334863 -0.02243496 -0.04773207 ... -0.10841947  0.03781788
  -0.06039597]
 [-0.05090242  0.03200291  0.08663241 ... -0.21919973  0.13552499
   0.23728875]
 ...
 [-0.83322743  0.54453521  0.23935086 ... -0.22277545  0.75050607
   0.12778346]
 [ 0.05702502  0.29606971 -0.19303012 ... -0.82667657  2.92369157
   0.36815178]
 [-0.450033    0.02318757 -0.25745796 ...  0.07824526  0.22051121
   0.37442113]]


In [25]:
DocIndexV1 = pd.DataFrame(vectors_w2v2)

In [26]:
IndexedPath = str(DirPpath.absolute()) + "\IndexedDataV1Tf.csv"
DocIndexV1.to_csv(IndexedPath)


Fine-tuning

In [75]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Setting the model with its hyperparameters
#model = Word2Vec(sentences = tokens, vector_size=300, min_count = 1)
model2 = Word2Vec(sentences = tokens, vector_size=300, min_count = 1)

In [62]:
model = KeyedVectors.load_word2vec_format(r"C:\Users\ecaudron001\Downloads\GoogleNews-vectors-negative300.bin",
                                         binary = True)

In [72]:
import numpy as np

In [78]:
model2.build_vocab(tokens)
total_examples = model2.corpus_count
model2.build_vocab([list(model2.wv.key_to_index.keys())], update=True)
model2.wv.intersect_word2vec_format(r"C:\Users\ecaudron001\Downloads\GoogleNews-vectors-negative300.bin", binary=True)
model2.train(tokens, total_examples=total_examples, epochs=model.iter)

IndexError: index 923 is out of bounds for axis 0 with size 1

In [ ]:
model2.wv.